# Supervised neural nets

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
%matplotlib inline

We're ready to build our first neural network. We will have multiple features we feed into our model, each of which will go through a set of perceptron models to arrive at a response which will be trained to our output.

Like many models we've covered, this can be used as both a regression or classification model.

First, we need to load our dataset. For this example we'll use The Museum of Modern Art in New York's [public dataset](https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv) on their collection.

In [34]:
artworks = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')

In [35]:
artworks.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

We'll also do a bit of data processing and cleaning, selecting columns of interest and converting URL's to booleans indicating whether they are present.

In [36]:
# Select Columns.
artworks = artworks[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

# Convert URL's to booleans.
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

# Drop films and some other tricky rows.
artworks = artworks[artworks['Department']!='Film']
artworks = artworks[artworks['Department']!='Media and Performance Art']
artworks = artworks[artworks['Department']!='Fluxus Collection']

# Drop missing data.
artworks = artworks.dropna()

In [37]:
artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.6000,168.9000
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.6401,29.8451
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.3000,31.8000
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.8000,50.8000
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.4000,19.1000


## Building a Model

Now, let's see if we can use multi-layer perceptron modeling (or "MLP") to see if we can classify the department a piece should go into using everything but the department name.

Before we import MLP from SKLearn and establish the model we first have to ensure correct typing for our data and do some other cleaning.

In [38]:
# Get data types.
artworks.dtypes

Artist           object
Nationality      object
Gender           object
Date             object
Department       object
DateAcquired     object
URL                bool
ThumbnailURL       bool
Height (cm)     float64
Width (cm)      float64
dtype: object

The `DateAcquired` column is an object. Let's transform that to a datetime object and add a feature for just the year the artwork was acquired.

In [39]:
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)
artworks['YearAcquired'] = artworks.DateAcquired.dt.year
artworks['YearAcquired'].dtype

dtype('int64')

Great. Let's do some more miscellaneous cleaning.

In [40]:
# Remove multiple nationalities, genders, and artists.
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_persons\)'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'

# Convert dates to start date, cutting down number of distinct examples.
artworks['Date'] = pd.Series(artworks.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]

# Final column drops and NA drop.
X = artworks.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

# Create dummies separately.
artists = pd.get_dummies(artworks.Artist)
nationalities = pd.get_dummies(artworks.Nationality)
dates = pd.get_dummies(artworks.Date)

# Concat with other variables, but artists slows this wayyyyy down so we'll keep it out for now
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates], axis=1)

Y = artworks.Department

In [41]:
# Alright! We've done our prep, let's build the model.
# Neural networks are hugely computationally intensive.
# This may take several minutes to run.

# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(1000,))
mlp.fit(X, Y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [42]:
mlp.score(X, Y)

0.7734033975588734

In [43]:
Y.value_counts()/len(Y)

Drawings & Prints        0.623627
Photography              0.226170
Architecture & Design    0.112928
Painting & Sculpture     0.033660
Media and Performance    0.003614
Name: Department, dtype: float64

In [44]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp, X, Y, cv=5)

array([0.6858811 , 0.66173949, 0.68257646, 0.61172382, 0.52736457])

Now we got a lot of information from all of this. Firstly we can see that the model seems to overfit, though there is still so remaining performance when validated with cross validation. This is a feature of neural networks that aren't given enough data for the number of features present. _Neural networks, in general, like_ a lot _of data_. You may also have noticed something also about neural networks: _they can take a_ long _time to run_. Try increasing the layer size by adding a zero. Feel free to interrupt the kernel if you don't have time...

Also note that we created bools for artist's name but left them out. Both of the above points are the reason for that. It would take much longer to run and it would be much more prone to overfitting.

## Model parameters

Now, before we move on and let you loose with some tasks to work on the model, let's go over the parameters.

We included one parameter: hidden layer size. Remember in the previous lesson, when we talked about layers in a neural network. This tells us how many and how big to make our layers. Pass in a tuple that specifies each layer's size. Our network is 1000 neurons wide and one layer. (100, 4, ) would create a network with two layers, one 100 wide and the other 4.

How many layers to include is determined by two things: computational resources and cross validation searching for convergence. It's generally less than the number of input variables you have.

You can also set an alpha. Neural networks like this use a regularization parameter that penalizes large coefficients just like we discussed in the advanced regression section. Alpha scales that penalty.

Lastly, we'll discuss the activation function. The activation function determines whether the output from an individual perceptron is binary or continuous. By default this is a 'relu', or 'rectified linear unit function' function. In the exercise we went through earlier we used this binary function, but we discussed the _sigmoid_ as a reasonable alternative. The _sigmoid_ (called 'logistic' by SKLearn because it's a 'logistic sigmoid function') allows for continuous variables between 0 and 1, which allows for a more nuanced model. It does come at the cost of increased computational complexity.

If you want to learn more about these, study [activation functions](https://en.wikipedia.org/wiki/Activation_function) and [multilayer perceptrons](https://en.wikipedia.org/wiki/Multilayer_perceptron). The [Deep Learning](http://www.deeplearningbook.org/) book referenced earlier goes into great detail on the linear algebra involved.

You could also just test the models with cross validation. Unless neural networks are your specialty cross validation should be sufficient.

For the other parameters and their defaults, check out the [MLPClassifier documentaiton](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier).

## Drill: Playing with layers

Now it's your turn. Using the space below, experiment with different hidden layer structures. You can try this on a subset of the data to improve runtime. See how things vary. See what seems to matter the most. Feel free to manipulate other parameters as well. It may also be beneficial to do some real feature selection work...

I will only use a subset of the data to speed up our runtime. The model runtime is extremely slow if the entire dataset is used. I will extract ~10% of the original data, or ~10000 rows, and then experiment with the different hidden layer structures. Below are the hidden layer structures I will try:
* Double 100,5 perceptron layer
* Double 100,5 perceptron layer and alpha of 1e-4
* Double 100,5 perceptron layer and alpha of 1e-6
* Double 1000,5 perceptron layer and alpha of 1e-6
* Triple 1000,50,10 perceptron layer and alpha of 1e-6
* Triple 500,500,100 perceptron layer and alpha of 1e-6
* Triple 100,1000,500 perceptron layer and alpha of 1e-6

In [45]:
# Your code here. Experiment with hidden layers to build your own model.
# We will only use a subset of the data to improve runtime.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.90)

In [46]:
X_train.head(100)

,URL,ThumbnailURL,Height (cm),Width (cm),YearAcquired,Gender_(),Gender_(Female),Gender_(Male),Gender_(male),Gender_\(multiple_persons\),...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
76484,False,False,10.600000,7.900000,1964,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
35835,True,False,19.900000,16.000000,1985,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6434,True,True,21.000000,7.600000,1977,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
84552,True,True,16.510000,22.860000,2005,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
62902,True,True,14.300000,17.800000,1948,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
65183,False,False,36.200000,41.900000,1949,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
121113,True,True,13.970028,21.907544,2014,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7739,True,True,83.800000,59.200000,2000,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
128786,False,False,57.150114,88.900178,2016,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76974,False,False,18.100000,23.600000,1971,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
Y_train.head()

76484              Photography
35835        Drawings & Prints
6434     Architecture & Design
84552        Drawings & Prints
62902        Drawings & Prints
Name: Department, dtype: object

In [48]:
# Establish and fit the model, with a double 100,5 perceptron layer.
mlp1 = MLPClassifier(hidden_layer_sizes=(100,5))
mlp1.fit(X_train, Y_train)
score = mlp1.score(X_train, Y_train)
print('Accuracy: {:.2%}'.format(score))

Accuracy: 62.87%


In [49]:
cross_val_score(mlp1, X_train, Y_train, cv=5)

array([0.62853173, 0.62853173, 0.69939787, 0.62911451, 0.62755102])

In [50]:
# Establish and fit the model, with a double 100,5 perceptron layer and alpha.
mlp2 = MLPClassifier(hidden_layer_sizes=(100,5), alpha=1e-4)
mlp2.fit(X_train, Y_train)
score = mlp2.score(X_train, Y_train)
print('Accuracy: {:.2%}'.format(score))

Accuracy: 62.87%


In [51]:
cross_val_score(mlp2, X_train, Y_train, cv=5)

array([0.62853173, 0.62853173, 0.62853173, 0.6286509 , 0.62894249])

In [52]:
# Establish and fit the model, with a double 100,5 perceptron layer and alpha.
mlp3 = MLPClassifier(hidden_layer_sizes=(100,5), alpha=1e-6)
mlp3.fit(X_train, Y_train)
score = mlp3.score(X_train, Y_train)
print('Accuracy: {:.2%}'.format(score))

Accuracy: 62.87%


In [53]:
cross_val_score(mlp3, X_train, Y_train, cv=5)

array([0.62899491, 0.62853173, 0.62853173, 0.62911451, 0.62894249])

In [54]:
# Establish and fit the model, with a double 1000,5 perceptron layer and alpha.
mlp4 = MLPClassifier(hidden_layer_sizes=(1000,5), alpha=1e-6)
mlp4.fit(X_train, Y_train)
score = mlp4.score(X_train, Y_train)
print('Accuracy: {:.2%}'.format(score))

Accuracy: 62.87%


In [55]:
cross_val_score(mlp4, X_train, Y_train, cv=5)

array([0.62853173, 0.62853173, 0.62853173, 0.62911451, 0.62894249])

In [56]:
# Establish and fit the model, with a triple 1000,50,10 perceptron layer and alpha.
mlp5 = MLPClassifier(hidden_layer_sizes=(1000,50,10), alpha=1e-6)
mlp5.fit(X_train, Y_train)
score = mlp5.score(X_train, Y_train)
print('Accuracy: {:.2%}'.format(score))

Accuracy: 64.43%


In [57]:
cross_val_score(mlp5, X_train, Y_train, cv=5)

array([0.22603057, 0.62853173, 0.62853173, 0.62911451, 0.62847866])

In [58]:
# Establish and fit the model, with a triple 500,500,100 perceptron layer and alpha.
mlp6 = MLPClassifier(hidden_layer_sizes=(500,500,100), alpha=1e-6)
mlp6.fit(X_train, Y_train)
score = mlp6.score(X_train, Y_train)
print('Accuracy: {:.2%}'.format(score))

Accuracy: 47.98%


In [59]:
cross_val_score(mlp6, X_train, Y_train, cv=5)

array([0.63362668, 0.65446966, 0.22927281, 0.6286509 , 0.25278293])

In [60]:
# Establish and fit the model, with a triple 100,1000,500 perceptron layer and alpha.
mlp7 = MLPClassifier(hidden_layer_sizes=(100,1000,500), alpha=1e-6)
mlp7.fit(X_train, Y_train)
score = mlp7.score(X_train, Y_train)
print('Accuracy: {:.2%}'.format(score))

Accuracy: 63.52%


In [61]:
cross_val_score(mlp7, X_train, Y_train, cv=5)

array([0.6373321 , 0.26771654, 0.66697545, 0.6550765 , 0.34461967])

# Observations
I noticed that increasing the visible layer size even by as much as tenfold (from 100 to 1000) did not increase the accuracy at all. Additionally, changing the alpha parameter had no effect on the accuracy. However, increasing the second layer size to 50 and adding a third layer of size 10 did result in a small improvement in accuracy. Interestingly, decreasing the visible layer size to 500 and increasing the hidden layer sizes tenfold to 500,100 resulted in a huge decrease in accuracy, indicating that perhaps more of an increase in the hidden layers is required to offset the decrease in the visible layer size. Decreasing the visible layer size further to 100 while increasing the hidden layer sizes to 1000,500 finally enabled me to reproduce a similar accuracy to what I had with a visible layer size of 1000.